# Computer simulations course 2018/2019-2 @ ELTE
# Assignment 3: Planetary Motions - Three-body problem
## 03.03.2019

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import sys
from scipy import stats
from datetime import datetime
import time
import imageio
import os, shutil
from matplotlib.patches import Circle
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [ ]:
sns.set_style(style='whitegrid')

### Planets

In [ ]:
# [Mass in kg,
#  Distance from central celestail body in AU,
#  eccentricity,
#  Size in AU]
Planets={
    'Sun': [1.989e30, 0, 0.0001, 4.649e-03],
    'Moon': [7.348e22, 0.00257, 0.0549, 1.161e-05],
    'Mercury': [3.285e23, 0.466697, 0.205630, 1.631e-05],
    'Venus': [4.867e24, 0.728213, 0.006772, 4.045e-05],
    'Earth': [5.972e24, 1.017, 0.0167086, 4.259e-05],
    'Mars': [6.39e23, 1.666, 0.0934, 2.266e-05],
    'Jupiter': [1.898e27, 5.4588, 0.0489, 4.673e-04],
    'Saturn': [5.683e26, 10.1238, 0.0565, 3.893e-04],
    'Uranus': [8.681e25, 20.11, 0.046381, 1.695e-04],
    'Neptune': [1.024e26, 30.33, 0.009456, 1.646e-04],
    'Pluto': [1.309e22, 49.305, 0.2488, 7.954e-06],
    'Halley': [2.2e14, 35.082, 0.96714, 3.68e-08]
}

Planet_Colors={
    'Sun': np.array([216, 148, 29])/255,
    'Moon': np.array([204, 198, 195])/255,
    'Mercury': np.array([186, 186, 186])/255,
    'Venus': np.array([216, 194, 153])/255,
    'Earth': np.array([45, 52, 130])/255,
    'Mars': np.array([217, 120, 62])/255,
    'Jupiter': np.array([187, 155, 99])/255,
    'Saturn': np.array([222, 181, 82])/255,
    'Uranus': np.array([201, 239, 241])/255,
    'Neptune': np.array([72, 120, 242])/255,
    'Pluto': np.array([65, 25, 20])/255,
    'Halley': np.array([0,0,0])/255
}

## Initial conditions

In [ ]:
choosen_planet_1 = 'Sun'
choosen_planet_2 = 'Jupiter'

# Gravitational constant [AU^3 * kg^-1 * year^-2]
G = 1.9838e-29

# Masses of choosen bodies [kg]
if(choosen_planet_1 != ''):
    m_1 = Planets[choosen_planet_1][0]
else:
    m_1 = Planets['Saturn'][0]

if(choosen_planet_2 != ''):
    m_2 = Planets[choosen_planet_2][0]
else:
    m_2 = Planets['Jupiter'][0]

# Eccentricity of choosen bodies
if(choosen_planet_1 != ''):
    ecc_1 = Planets[choosen_planet_1][2]
else:
    ecc_1 = 0.6
if(choosen_planet_2 != ''):
    ecc_2 = Planets[choosen_planet_2][2]
else:
    ecc_2 = 0.8

# Distance of the choosen bodies' center of mass [AU]
if(choosen_planet_2 != ''):
    r_dist = Planets[choosen_planet_2][1]
else:
    r_dist = 0.1

# Step size
dt = 5 * 1e-2
# Adaptive accuracy of simulation
accuracy = 1e-12

# Calculated orbit parameters
# r_ap: Apogee distance; measured from the system's center of mass
# a: semi-major axis in [AU]
# b: semi-minor axis in [AU]
r_ap_1 = m_2/(m_1+m_2) * r_dist
r_ap_2 = m_1/(m_1+m_2) * r_dist
a_1 = r_ap_1 / (1 + ecc_1)
a_2 = r_ap_2 / (1 + ecc_2)
b_1 = np.sqrt(1 - ecc_1**2) * a_1
b_2 = np.sqrt(1 - ecc_2**2) * a_2

# Velocities in the apogee [AU/year]
v0_1 = np.sqrt(G * m_2**3/(m_1 + m_2)**2 * (2 / r_ap_1 - 1 / a_1))  # Initial velocity of first body (tangential along y-axis) [AU/day]
v0_2 = np.sqrt(G * m_1**3/(m_1 + m_2)**2 * (2 / r_ap_2 - 1 / a_2))  # Initial velocity of second body (tangential along y-axis) [AU/day]

# Orbital period in [year]
T = np.sqrt(4 * np.pi * np.pi * np.power(r_dist,3)/ (G * (m_1 + m_2)))

# Number of years to plot
plotting_years = 5 * T

# Others
steps = 1
fps = 40
image_dpi = 150
image_format = 'pdf'
image_path = '..\\Documentation\\src\\images\\three_body\\'

In [ ]:
print('r_ap_1:', r_ap_1)
print('r_ap_2:', r_ap_2)

print('a_1:', a_1)
print('a_2:', a_2)

print('T:', T)

print('v0_1:', v0_1)
print('v0_2:', v0_2)

## Generating small bodies

In [ ]:
def sign_choose():
    return -1 if random.random() < 0.5 else 1

In [ ]:
half_number_of_bodies = 50

coordinates_array = np.zeros((2*half_number_of_bodies, 5))

nrows = 1
ncols = 1
picsize = 20
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*(picsize)*(a_2/b_2),nrows*picsize))

x_scatter = 0.1
y_scatter = 0.1
for i in range(0, half_number_of_bodies):
    
    # X coordinates
    coordinates_array[i,1] = random.random() * a_2 * sign_choose()
    # Y coordinates
    coordinates_array[i,2] = b_2/a_2 * np.sqrt(a_2**2 - coordinates_array[i,1]**2) * sign_choose()
    
for i in range(half_number_of_bodies, 2*half_number_of_bodies):
    
    # Y coordinates
    coordinates_array[i,2] = random.random() * b_2 * sign_choose()
    # X coordinates
    coordinates_array[i,1] = a_2/b_2 * np.sqrt(b_2**2 - coordinates_array[i,2]**2) * sign_choose()
    
# Scatter particles
norm = 2.3
for i in range(0, 2*half_number_of_bodies):
    
    coordinates_array[i,0] = random.random() * 1e15
    coordinates_array[i,1] += random.random() * x_scatter * a_2 * sign_choose() - (a_2 - (1+ecc_2) * a_2)
    coordinates_array[i,2] += random.random() * y_scatter * b_2 * sign_choose()
    
    # X velocities
    if(coordinates_array[i,2] < 0):
        coordinates_array[i,3] = -np.sqrt(np.abs(4 * np.pi * np.pi * (2/coordinates_array[i,1]) - 1/a_2))/norm
    else:
        coordinates_array[i,3] = np.sqrt(np.abs(4 * np.pi * np.pi * (2/coordinates_array[i,1]) - 1/a_2))/norm
    
    # Y velocities
    if(coordinates_array[i,1] < 0):
        coordinates_array[i,4] = np.sqrt(np.abs(4 * np.pi * np.pi * (2/coordinates_array[i,2]) - 1/a_2))/norm
    else:
        coordinates_array[i,4] = -np.sqrt(np.abs(4 * np.pi * np.pi * (2/coordinates_array[i,2]) - 1/a_2))/norm

# Save to dat file
np.savetxt('small_objects.dat', coordinates_array, delimiter='\t')
    
axes.scatter(coordinates_array[::steps,1], coordinates_array[::steps,2])
axes.scatter(coordinates_array[::steps,3], coordinates_array[::steps,4])
plt.show()

## Run simulation

### Command line arguments

In [ ]:
def mode_choose(mode, odeint, relat):
    
    current_mode = ('..\Release\kepler_three.exe ' +
                    mode + ' ' +
                    odeint + ' ' +
                    relat + ' ' +
                    str(m_1) + ' ' +
                    str(m_2) + ' ' +
                    str(r_dist) + ' ' +
                    str(ecc_1) + ' ' +
                    str(ecc_2) + ' ' +
                    str(plotting_years) + ' ' +
                    str(dt) + ' ' +
                    str(accuracy) + ' ' +
                    str(half_number_of_bodies * 2)
                   )

    return(current_mode)

In [ ]:
current_mode = mode_choose(mode = 'fixed', odeint='runge', relat='no')
os.system(current_mode)

In [ ]:
data_fixed = np.genfromtxt('fixed.dat')

In [ ]:
data_fixed.shape

In [ ]:
data_small = np.genfromtxt('fixed_smalls.dat')

In [ ]:
data_small.shape

In [ ]:
nrows = 1
ncols = 2
picsize = 20
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*(picsize)*(a_2/b_2),nrows*picsize))

axes[0].add_patch(Circle(xy=(r_ap_1, 0), radius=Planets[choosen_planet_1][3], fc=Planet_Colors[choosen_planet_1], zorder=10))
axes[1].add_patch(Circle(xy=(r_ap_1, 0), radius=Planets[choosen_planet_1][3], fc=Planet_Colors[choosen_planet_1], zorder=10))
axes[0].add_patch(Circle(xy=(r_ap_1, 0), radius=Planets[choosen_planet_1][3], fc=Planet_Colors[choosen_planet_1], zorder=10))
axes[1].add_patch(Circle(xy=(r_ap_1, 0), radius=Planets[choosen_planet_1][3], fc=Planet_Colors[choosen_planet_1], zorder=10))

axes[0].plot(data_fixed[::steps,1], data_fixed[::steps,2], c='red')
axes[0].plot(data_fixed[::steps,6], data_fixed[::steps,7], c='green', lw=5)
axes[1].plot(data_fixed[::steps,1], data_fixed[::steps,2], c='red')
axes[1].plot(data_fixed[::steps,6], data_fixed[::steps,7], c='green', lw=5)

maximum_range = 15

axes[0].set_xlim(-maximum_range,maximum_range)
axes[0].set_ylim(-maximum_range,maximum_range)
axes[1].set_xlim(-10*maximum_range,10*maximum_range)
axes[1].set_ylim(-10*maximum_range,10*maximum_range)

for i in range(0,half_number_of_bodies*2):
    if(max(np.abs(data_small[::steps,1+i*5])) > maximum_range or
       max(np.abs(data_small[::steps,2+i*5])) > maximum_range):
        axes[1].plot(data_small[::steps,1+i*5],data_small[::steps,2+i*5])
    else:
        axes[0].plot(data_small[::steps,1+i*5],data_small[::steps,2+i*5])

axes[0].set_title('Fixed step propagation\nStable Bodies', fontsize=40)
axes[1].set_title('Fixed step propagation\nUnstable Bodies', fontsize=40)

axes[0].set_xlabel('Distance from center of mass along X [AU]', fontsize=40)
axes[0].set_ylabel('Distance from center of mass along Y [AU]', fontsize=40)
axes[1].set_xlabel('Distance from center of mass along X [AU]', fontsize=40)
axes[1].set_ylabel('Distance from center of mass along Y [AU]', fontsize=40)

axes[0].tick_params(axis='both', which='major', labelsize=30)
axes[1].tick_params(axis='both', which='major', labelsize=30)

legend_elements = [Line2D([0], [0], color='red', lw=1, label='Orbit of {0}'.format(choosen_planet_1)),
                   Line2D([0], [0], color='green', lw=5, label='Orbit of {0}'.format(choosen_planet_2)),
                   Line2D([0], [0], marker='o', color='white', markerfacecolor=Planet_Colors[choosen_planet_1],
                          markersize=10, label=choosen_planet_1),
                   Line2D([0], [0], marker='o', color='white', markerfacecolor=Planet_Colors[choosen_planet_2],
                          markersize=10, label=choosen_planet_2)]

axes[0].legend(handles=legend_elements, fontsize=30)
axes[1].legend(handles=legend_elements, fontsize=30)
fig.tight_layout()
plt.savefig(image_path +
            choosen_planet_2 + '_around_' + choosen_planet_1 + '_fixed.' +
            image_format,
            format=image_format,
            dpi=image_dpi,
            bbox_inches='tight')
plt.show()

In [ ]:
'''ffmpeg -i input1.mp4 -c copy -bsf:v h264_mp4toannexb -f mpegts intermediate1.ts
ffmpeg -i input2.mp4 -c copy -bsf:v h264_mp4toannexb -f mpegts intermediate2.ts
ffmpeg -i "concat:intermediate1.ts|intermediate2.ts" -c copy -bsf:a aac_adtstoasc output.mp4''';